# PyTerrier ANCE Demo Notebook - Vaswani

This notebook demonstrates use of [PyTerrier plugin for ANCE](https://github.com/terrierteam/pyterrier_ance) for dense passage retrieval. 

[ANCE](https://github.com/microsoft/ANCE) is a dense retrieval system leveraging single representations to encode documents and queries. ANCE does not require combination with sparse retrieval. ANCE leverages a training mechanism that constructs negatives from an Approximate Nearest Neighbor (ANN) index of the corpus, which is parallelly updated with the learning process to select more realistic negative training instances than the negative training instances selected by a sparse retrieval mechanism.

ANCE is built on top of [BERT](https://arxiv.org/abs/1810.04805), and it nearly matches the accuracy of sparse retrieval and BERT reranking using dot-product in the ANCE-learned representation space and provides almost 100x speed-up.

The corpus used in this demo is the [Vaswani NPL corpus](http://ir.dcs.gla.ac.uk/resources/test_collections/npl/), a corpus of 11,429 scientific abstract, with corresponding queries and relevance assessments.

## Installation 

We need to install [PyTerrier](https://github.com/terrier-org/pyterrier).

In [ ]:
!pip install python-terrier

[ANCE](https://github.com/microsoft/ANCE) requires [FAISS](https://github.com/facebookresearch/faiss), a library for efficient similarity search and clustering of dense vectors.

This is the setup for FAISS on Colab. YMMV outside of Colab.

In [ ]:
!apt install libomp-dev
!pip install faiss

This installs the [PyTerrier plugin for ANCE](https://github.com/terrierteam/pyterrier_ance). It supplies an indexer and a retrieval transformer. This also installs [ANCE](https://github.com/microsoft/ANCE).

In [ ]:
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_ance.git

# Setup

Lets get [PyTerrier](https://github.com/terrier-org/pyterrier) started. This will download the latest version of the [Terrier](http://terrier.org) IR platform.

In [ ]:
import pyterrier as pt
pt.init(tqdm='notebook')

We are using the [Vaswani dataset](http://ir.dcs.gla.ac.uk/resources/test_collections/npl/) – lets collect the topics & qrels.

In [ ]:
dataset = pt.get_dataset("irds:vaswani")

This downloads the model checkpoint listed on the [ANCE github repository](https://github.com/microsoft/ANCE/#results). Download time can vary, on average it requires 11-12 minutes.

In [ ]:
import os
if not os.path.exists("Passage_ANCE_FirstP_Checkpoint.zip"):
  !wget https://webdatamltrainingdiag842.blob.core.windows.net/semistructstore/OpenSource/Passage_ANCE_FirstP_Checkpoint.zip
  !unzip Passage_ANCE_FirstP_Checkpoint.zip

## Indexing

This indexes the [Vaswani dataset](http://ir.dcs.gla.ac.uk/resources/test_collections/npl/). Indexing takes about 3 minutes using a Colab GPU.

In [ ]:
!rm -rf /content/anceindex

import pyterrier_ance
indexer = pyterrier_ance.ANCEIndexer(checkpoint_path="/content/Passage ANCE(FirstP) Checkpoint",
                                     index_path"/content/anceindex",
                                     num_docs=11429)
indexer.index(dataset.get_corpus_iter())

We will not need the indexer anymore, so we free up some memory.

In [ ]:
del(indexer)

The indexing procedure generates a number of [FAISS](https://github.com/facebookresearch/faiss) shards, together with some additional files.

In [ ]:
!ls /content/anceindex

# Retrieval

Now that indexing has completed, we can load in the index and the checkpoint model (which we will need for encoding queries). Index loading can take some times, as the [FAISS](https://github.com/facebookresearch/faiss) shards need to be loaded in main memory.

In [ ]:
ance_retr = pyterrier_ance.ANCERetrieval(checkpoint_path="/content/Passage ANCE(FirstP) Checkpoint",
                                        index_path="/content/anceindex")

Here we can ask [PyTerrier](https://github.com/terrier-org/pyterrier) to search the [ANCE](https://github.com/microsoft/ANCE) index for `'chemical reactions'`, returning the top 10 relevant documents.

In [ ]:
(ance_retr % 10).search("chemical reactions")

# Running an Experiment

Lets prepare an experiment. Firstly, lets create in a BM25 baseline transformer.

In [ ]:
bm25 = pt.BatchRetrieve(pt.get_dataset("vaswani").get_index(), wmodel="BM25")

Finally, lets evaluate our performance. We also load in an BM25 index for the same corpus for comparison reasons.

In [ ]:
pt.Experiment(
    [bm25, ance_retr], 
    dataset.get_topics(), 
    dataset.get_qrels(), 
    eval_metrics=["map", "recip_rank", "mrt"]
    )

So on this collection, ANCE isnt as effective under MAP or MRR, but it does have a lower mean response time.